In [1]:
import json
from matplotlib import pyplot as plt
from collections import defaultdict
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
import sklearn
import numpy as np
import pandas as pd
import random
import gzip
import math
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy as np
import string
import random
import string
import datetime
from sklearn import linear_model
import csv
import pandas as pd
from sklearn.metrics import mean_squared_error
import sys

In [2]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)


def SSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) 



In [3]:
df = pd.read_json('Amazon_Fashion.jsonl.gz', compression='gzip', lines = True)

#'df = pd.read_json('Amazon_Fashion.jsonl.gz', compression='gzip', lines = True, chunksize=20) # damit ich nicht so viel ram verbrauche
# df = next(df)

df = df.sort_values(by = "timestamp").drop_duplicates(subset= ["user_id", "asin"], keep = "last")

allRatings = df.get(["user_id", "asin", "rating"])
# df = df.sample(df.shape[0], replace= False) #shuffle data through sampling without replacement

try: 
    min_reviews = int(sys.argv[1]) 
except:
    min_reviews = 0
allRatings_filtered = allRatings[allRatings['user_id'].map(allRatings['user_id'].value_counts()) > min_reviews] 

# allRatings_numpy = allRatings_filtered.to_numpy()



# training_size = int(allRatings_numpy.size * 0.9)
# test_size = allRatings_numpy.size - training_size

# ratingsTrain = allRatings_numpy[:training_size]
# ratingsValid = allRatings_numpy[-test_size:]

ratingsTrain, ratingsValid = train_test_split(allRatings_filtered.to_numpy(), test_size=0.1, shuffle=True)

ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))



N = ratingsTrain.shape[0]
nUsers = len(ratingsPerUser)
nItems = len(ratingsPerItem)
users = list(ratingsPerUser.keys())
items = list(ratingsPerItem.keys())


alpha_base = sum([r for _, _, r in ratingsTrain]) / ratingsTrain.shape[0]
alpha = alpha_base

userBiases = defaultdict(float)
itemBiases = defaultdict(float)


def prediction(user, item):
    return alpha + userBiases.get(user, 0) + itemBiases.get(item, 0)


def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    alpha = theta[0]
    userBiases = dict(zip(users, theta[1:nUsers+1]))
    itemBiases = dict(zip(items, theta[1+nUsers:]))

def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(d[0], d[1]) for d in ratingsTrain]
    cost = SSE(predictions, labels)
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for i in itemBiases:
        cost += lamb*itemBiases[i]**2
    return cost

def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(ratingsTrain)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    for u,i,r in ratingsTrain:
        pred = prediction(u, i)
        diff = pred - r
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[i] += 2/N*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
    for i in itemBiases:
        dItemBiases[i] += 2*lamb*itemBiases[i]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
    return np.array(dtheta)



output_file = open(f"./more_than_{min_reviews}_output_{datetime.datetime.now()}", "w")

labels = [r for _, _, r in ratingsTrain]
for i in np.power(10., np.arange(-6, -3,1)):
    alpha = alpha_base
    userBiases = defaultdict(float)
    itemBiases = defaultdict(float)
    lamb = i
    result = scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0] * (nUsers + nItems), derivative, args=(labels, lamb))
    unpack(result[0])

    validMSE = sum((prediction(u,b) - r) ** 2 for u, b, r in ratingsValid) /len(ratingsValid)
    print(f"lambda: {lamb} MSE: {validMSE}")
    output_file.write(f"lambda: {lamb} MSE: {validMSE}\n")

output_file.close()  

KeyboardInterrupt: 

# Rating Analysis 4

## Details
- All with duplicates filtered out
- use of the `test_train_split` function by `sklearn`
- see results in folder 'run'

## Test runs
### Condition: more_than_0
|   Lambda |    run1 |    run2 |    run3 |
|---------:|--------:|--------:|--------:|
|   1e-06  | 1.99971 | 2.03132 | 2.02951 |
|   1e-05  | 2.03468 | 1.92611 | 1.92139 |
|   0.0001 | 2.00528 | 2.03132 | 1.99967 |

### Condition: more_than_1
|   Lambda |    run1 |    run2 |    run3 |
|---------:|--------:|--------:|--------:|
|   1e-06  | 1.78582 | 1.79589 | 1.79792 |
|   1e-05  | 1.78778 | 1.74401 | 1.72535 |
|   0.0001 | 1.79751 | 1.8231  | 1.80963 |

### Condition: more_than_2
|   Lambda |    run1 |    run2 |    run3 |
|---------:|--------:|--------:|--------:|
|   1e-06  | 1.7136  | 1.71797 | 1.70042 |
|   1e-05  | 1.57232 | 1.57365 | 1.56407 |
|   0.0001 | 1.70294 | 1.70694 | 1.64227 |

### Condition: more_than_3
|   Lambda |    run1 |    run2 |    run3 |
|---------:|--------:|--------:|--------:|
|   1e-06  | 1.6984  | 1.71788 | 1.646   |
|   1e-05  | 1.47606 | 1.48537 | 1.50144 |
|   0.0001 | 1.49987 | 1.48979 | 1.5185  |

### Condition: more_than_4
|   Lambda |    run1 |    run2 |    run3 |
|---------:|--------:|--------:|--------:|
|   1e-06  | 1.58972 | 1.52586 | 1.58443 |
|   1e-05  | 1.44104 | 1.44257 | 1.40162 |
|   0.0001 | 1.39766 | 1.41456 | 1.37749 |
